In [1]:
import sys
from copy import deepcopy

def distance(loc1, loc2):
    return ((loc1[0] - loc2[0])**2 + (loc1[1] - loc2[1])**2)**0.5

In [2]:
class RideSharing:

    requests = []
    tours = []

    def add_request(self, pickup_loc, dropoff_loc, request_time, max_waiting_duration):
        self.requests.append(Request(pickup_loc, dropoff_loc, request_time, max_waiting_duration))

    def remove_request(self, request):
        if request in self.requests:
            self.requests.remove(request)

    def add_tour(self, depot_loc, latest_return=sys.maxsize):
        self.tours.append(Tour(depot_loc, latest_return))

    def remove_tour(self, tour):
        if tour in self.tours:
            self.tours.remove(tour)

    def solve(self):
        parallel_insertion(self.tours, self.requests)

    def __init__(self):
        pass

$L$:  locations ($l \in L$)

- *loc:* Koordinaten der Location
- *request:* Zugehöriges Request zur Location

$o_r$: Pick-Ups ($o_r \in L_o \subset L$)

- erbt von Location

$d_r$: Drop-Offs ($d_r \in L_d \subset L$)

- erbt von Location

In [3]:
class Location:

    def __init__(self, loc, request):
        self.loc = loc
        self.request = request


class PickUp(Location):
    is_pickup = True
    is_dropoff = False

    def __init__(self, loc, request):
        super().__init__(loc, request)


class DropOff(Location):
    is_pickup = False
    is_dropoff = True

    def __init__(self, loc, request):
        super().__init__(loc, request)

$R$:  requests ($r \in R$)

- receiving time $t_r$
- origin $o_r \in L$
- destination $d_r \in L$
- time window $[b_r, e_r]$
    - $b_r = t_r$
    - $e_r = t_r + c_{o_r,d_r} + \alpha$

In [4]:
class Request:

    count = 0

    def __init__(self, pickup_loc, dropoff_loc, request_time, max_waiting_duration):
        self.id = Request.count
        Request.count += 1
        self.request_time = request_time
        self.max_waiting_duration = max_waiting_duration
        self.latest_dropoff = request_time + distance(pickup_loc, dropoff_loc) + max_waiting_duration
        self.pickup = PickUp(pickup_loc, self)
        self.dropoff = DropOff(dropoff_loc, self)

$V$:  vehicles ($v \in V$)

In [5]:
class Tour:

    tour = []
    early = []
    late = []
    length = 0
    total_cost = 0

    def __init__(self, depot_loc, latest_return=sys.maxsize):
        depot = Request(depot_loc, depot_loc, 0, latest_return)
        self.tour += [depot.pickup, depot.dropoff]
        self.early += [0, 0]
        self.late += [latest_return, latest_return]
        self.length += 2

    def get_total_cost(self):
        return self.total_cost

    def feasible(self, place, i):
        if place.is_dropoff and place.request.pickup not in self.tour[:i]:
            return False
        if place.is_pickup and place.request.dropoff in self.tour[:i]:
            return False
        e, l = self.insertion_time_window(place, i)
        return e <= l

    def insertion_cost(self, place, i):
        return (distance(self.tour[i-1].loc, place.loc) + distance(place.loc, self.tour[i].loc)
                - distance(self.tour[i-1].loc, self.tour[i].loc))

    def insertion_time_window(self, place, i):
        e = max(place.request.request_time + distance(self.tour[i-1].loc, place.loc), self.early[i-1] + distance(self.tour[i-1].loc, place.loc))
        l = min(place.request.latest_dropoff, self.late[i] - distance(place.loc, self.tour[i].loc))
        return e, l

    def insert(self, place, i):
        self.total_cost += self.insertion_cost(place, i)
        self.tour.insert(i, place)
        self.length += 1
        self.update(i)

    def update(self, i):
        e, l = self.insertion_time_window(self.tour[i], i)
        self.early.insert(i, e)
        self.late.insert(i, l)
        # Update der Zeitfenster (late) der vorigen Orte
        for k in range(i-1, -1, -1):
            self.late[k] = min(self.late[k], self.late[k+1] - distance(self.tour[k].loc, self.tour[k+1].loc))
        # Update der Zeitfenster (early) der nachfolgenden Orte
        for k in range(i+1, self.length):
            self.early[k] = max(self.early[k], self.early[k-1] + distance(self.tour[k-1].loc, self.tour[k].loc))

    def printable(self):
        result= []
        for i in range(self.length):
            if i == 0 or i == self.length-1:
                typ = "Depot"
            elif self.tour[i].is_pickup:
                typ = "Pick-Up"
            else:
                typ = "Drop_Off"
            result.append((self.tour[i].request.id, self.tour[i].loc, self.early[i], typ))
        return result

In [6]:
def parallel_insertion(tours, requests, M=sys.maxsize):
    reject = []
    # Kunden = copy.deepcopy(fiktivekunden)

    while requests:
        pOpt = M
        for j in range(len(requests)):
            for k in range(len(tours)):
                # spos: Position in Tour k, an der der Pickup des Kunden eingefügt wird
                for spos in range(1, tours[k].length):
                    if tours[k].feasible(requests[j].pickup, spos):
                        skosten = tours[k].insertion_cost(requests[j].pickup, spos)

                        # speichert die aktuell gültige Tour in einer temporären Variablen
                        tour_temp = deepcopy(tours[k])

                        # füge den Pickup des Kunden vorübergehend in den Tourenplan ein und Update der Zeitfenster
                        tour_temp.insert(requests[j].pickup, spos)

                        # Prüfe für alle Positionen im Tourenplan nach dem eingefügten Pickup ob es einen möglichen Drop-off gibt
                        for epos in range(spos + 1, tour_temp.length):
                            if tour_temp.feasible(requests[j].dropoff, epos):
                                ekosten = tour_temp.insertion_cost(requests[j].dropoff, epos)
                                if skosten + ekosten < pOpt:
                                    reqOpt = requests[j]
                                    tourOpt = tours[k]
                                    sposOpt = spos
                                    eposOpt = epos
                                    pOpt = skosten + ekosten

        if pOpt < M:
            # Setze den Pickup und Drop-Off des Kunden ein und aktualisiere Zeitfenster
            tourOpt.insert(reqOpt.pickup, sposOpt)
            tourOpt.insert(reqOpt.dropoff, eposOpt)

            # Entferne den eigesetzten Kunden aus der Liste der noch ausstehenden Kunden
            requests.remove(reqOpt)
        else:
            # Wenn kein Kunde den Feasibility mehr besteht, speichere die abgewiesenen Kunden und leere die Liste
            reject = deepcopy(requests)
            requests = []

    return tours, reject

In [7]:
instance = RideSharing()
instance.add_tour((1, 1))
instance.add_tour((6, 6))
instance.add_request((1, 8), (5, 9), 1, 10)
instance.add_request((6, 5), (3, 4), 0, 10)
instance.add_request((3, 2), (8, 5), 0, 10)
instance.solve()

([<__main__.Tour at 0x288258b7da0>, <__main__.Tour at 0x288258b7d30>],
 [<__main__.Request at 0x288258c2b00>])

In [8]:
RideSharing.tours[1].tour

In [9]:
RideSharing.tours[0].printable()

[(0, (1, 1), 0, 'Depot'),
 (4, (3, 2), 2.23606797749979, 'Pick-Up'),
 (3, (6, 5), 6.478708664619075, 'Pick-Up'),
 (3, (3, 4), 9.640986324787454, 'Drop_Off'),
 (4, (8, 5), 14.740005838380238, 'Drop_Off'),
 (0, (1, 1), 22.802263586678787, 'Depot')]

In [10]:
RideSharing.tours[1].printable()

[(0, (1, 1), 0, 'Depot'), (4, (3, 2), 2.23606797749979, 'Depot')]